In [1]:
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
import os
from itertools import product, combinations

from paths import *
from ssd_paths import *
from misc_recorder import *

import shutil

In [3]:
hyper_dir = model_save_
ts = "0206170859"
plot_save_place = "../plots/"
mk(plot_save_place)

In [4]:
for run_case in range(1, 11): 
    dir = os.path.join(hyper_dir, f"H18-{ts}-{run_case}", "large")
    for runtype, cv in list(product(["lf", "hf", "ff"], ["c", "v"])):
        shutil.copy(os.path.join(dir, cv, runtype, "vis.png"), os.path.join(plot_save_place, f"{run_case}-{cv}-{runtype}-vis.png"))

In [3]:
class ResReader: 
    def __init__(self, dir, runtype="ff", cv="full"):
        self.dir = dir
        self.runtype = runtype
        self.cv = cv
    
    def read(self):
        read_dir = os.path.join(self.dir, self.cv, self.runtype)
        train = ListRecorder(os.path.join(read_dir, 'train.acc'))
        valid = ListRecorder(os.path.join(read_dir, f'valid.acc'))
        fullvalid = ListRecorder(os.path.join(read_dir, f'full_valid.acc'))
        trainlikevalid = ListRecorder(os.path.join(read_dir, f'trainlikevalid.acc'))
        train.read()
        valid.read()
        fullvalid.read()
        trainlikevalid.read()
        return train.get(), valid.get(), fullvalid.get(), trainlikevalid.get()

In [4]:
def draw(reses, y_label="Accuracy", title=""): 
    data = list(reses.values())

    # Function to calculate mean and 95% CI
    def mean_confidence_interval(data, confidence=0.95):
        n = len(data)
        mean = np.mean(data)
        sem = stats.sem(data)
        margin = sem * stats.t.ppf((1 + confidence) / 2., n-1)
        return mean, margin

    # Calculate means and CIs
    means = [mean_confidence_interval(group)[0] for group in data]
    cis = [mean_confidence_interval(group)[1] for group in data]

    # Plotting
    fig, ax = plt.subplots()
    x_pos = np.arange(len(data))
    ax.bar(x_pos, means, yerr=cis, align='center', alpha=0.7, ecolor='black', capsize=10)
    ax.set_ylabel(y_label)
    ax.set_xticks(x_pos)
    ax.set_xticklabels(reses.keys())
    ax.set_title(title)
    ax.yaxis.grid(True)

    # Show the plot
    plt.tight_layout()
    # plt.show()
    plt.savefig(os.path.join(plot_save_place, f"{ts}-{title}.png"))